In [ ]:
import speech_recognition as sr
import pyttsx3
import openai
from pingpongthread import PingPongThread
from time import sleep
from keyboard import is_pressed

In [ ]:
# 로봇을 연결하는 인스턴스 생성.
PingPong = PingPongThread(number=2, group_id=46)
# 로봇 제어 쓰레드 시작.
PingPong.start()

Found device: USB 직렬 장치(COM3)
Serial connected.
Write data: FF FF 46 FF 20 00 AD 00 0B 1A 46


In [ ]:
# 오디오 장치의 리스트를 확인.
PingPong.get_audio_list()

['Microsoft Sound Mapper - Input',
 '마이크(Realtek(R) Audio)',
 'Microsoft Sound Mapper - Output',
 '스피커(Realtek(R) Audio)',
 '주 사운드 캡처 드라이버',
 '마이크(Realtek(R) Audio)',
 '주 사운드 드라이버',
 '스피커(Realtek(R) Audio)',
 '스피커(Realtek(R) Audio)',
 '마이크(Realtek(R) Audio)',
 '스테레오 믹스 (Realtek HD Audio Stereo input)',
 'Speakers 1 (Realtek HDA Primary output with SST)',
 'Speakers 2 (Realtek HDA Primary output with SST)',
 'PC 스피커 (Realtek HDA Primary output with SST)',
 'Headphones 1 (Realtek HD Audio 2nd output with SST)',
 'Headphones 2 (Realtek HD Audio 2nd output with SST)',
 'PC 스피커 (Realtek HD Audio 2nd output with SST)',
 '마이크 (Realtek HD Audio Mic input)']

Connected with a master robot.
Connected robots: 2
Fully connected.


In [ ]:

# OpenAI API 키 입력
openai.api_key = 'your-openai-api-key'

# 음성 합성 엔진 초기화
engine = pyttsx3.init()

def speak(text):
    engine.say(text)
    engine.runAndWait()

def listen():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening...")
        audio = r.listen(source)
    try:
        text = r.recognize_google(audio, language='ko-KR')
        print(f"👂 Heard: {text}")
        return text
    except sr.UnknownValueError:
        return "음성을 인식하지 못했어요."
    except sr.RequestError:
        return "음성 인식 오류가 발생했습니다."

def ask_chatgpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

# 핑퐁 로봇 시작 (한 번만 실행)
robot = PingPongThread()
robot.start()


In [ ]:
# 음성 듣기
command = listen()

# GPT에게 질문
response = ask_chatgpt(command)

# 대답
speak(response)
print(f"🤖 ChatGPT: {response}")

# 로봇 제어 예시
if "왼쪽" in command:
    robot.move_left()
elif "오른쪽" in command:
    robot.move_right()
elif "멈춰" in command:
    robot.stop()
elif "종료" in command:
    speak("로봇을 종료합니다.")
    robot.stop()
